In [20]:
import os
import pandas as pd
from pathlib import Path

# Set project root and define directories
project_root = Path(...).resolve()
os.chdir(project_root)

data_dir = project_root / "01_scraping"
output_dir = project_root / "02_preprocessing"
output_dir.mkdir(parents=True, exist_ok=True)

# Load La Liga data
la_liga_data_path = data_dir / "la_liga_data.csv"
la_liga_data = pd.read_csv(la_liga_data_path)

# Function to compute rolling averages
def rolling_averages(group, cols, new_cols):
    group = group.sort_values("Date")
    rolling_stats = group[cols].rolling(3, closed='left').mean()
    group[new_cols] = rolling_stats
    group = group.dropna(subset=new_cols)
    return group

# Define columns for rolling averages
cols = ["GF", "GA", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]
new_cols = [f"{c}_Rolling" for c in cols]

# Split recent and past data
recent_data = la_liga_data[la_liga_data["Season"] == "2024-2025"]
recent_data = recent_data.groupby("Team").apply(rolling_averages, cols=cols, new_cols=new_cols).reset_index(drop=True)
past_data = la_liga_data[la_liga_data["Season"] != "2024-2025"]

# Combine past and recent data
combined_data = pd.concat([past_data, recent_data], ignore_index=True)

# Feature engineering
combined_data["Date"] = pd.to_datetime(combined_data["Date"])
combined_data["Venue_Code"] = combined_data["Venue"].astype("category").cat.codes
combined_data["Opp_Code"] = combined_data["Opponent"].astype("category").cat.codes
combined_data["Hour"] = combined_data["Time"].str.replace(":.+", "", regex=True)
combined_data["Day_Code"] = combined_data["Date"].dt.dayofweek
combined_data["Target"] = (combined_data["Result"] == "W").astype("int")

# Clean Data
combined_data["Team"] = combined_data["Team"].str.replace("-", " ")
combined_data["Opponent"] = combined_data["Opponent"].str.replace("-", " ")

# Clean Data
combined_data["Team"] = combined_data["Team"].replace({
    "Alavés": "Alaves",
    "Cádiz": "Cadiz",
    "Atlético Madrid": "Atletico Madrid",
    "Leganés" : "Leganes",
})
combined_data["Opponent"] = combined_data["Opponent"].replace({
    "Alavés": "Alaves",
    "Cádiz": "Cadiz",
    "Atlético Madrid": "Atletico Madrid",
    "Leganés" : "Leganes"
})

# Save combined_data.csv
combined_data_path = output_dir / "combined_data.csv"
combined_data.to_csv(combined_data_path, index=False)

combined_data

/var/folders/cb/fcr_jbhj7x9fd02p67m1jzmr0000gn/T/ipykernel_95342/3773142065.py:32: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  recent_data = recent_data.groupby("Team").apply(rolling_averages, cols=cols, new_cols=new_cols).reset_index(drop=True)


,Team,Season,Date,Time,Round,Day,Venue,Result,GF,GA,...,SoT_Rolling,Dist_Rolling,FK_Rolling,PK_Rolling,PKatt_Rolling,Venue_Code,Opp_Code,Hour,Day_Code,Target
0,Alaves,2023-2024,2023-08-14,19:30,Matchweek 1,Mon,Away,L,0,1,...,NaN,NaN,NaN,NaN,NaN,0,7,19,0.0,0
1,Alaves,2023-2024,2023-08-21,19:00,Matchweek 2,Mon,Home,W,4,3,...,NaN,NaN,NaN,NaN,NaN,1,23,19,0.0,1
2,Alaves,2023-2024,2023-08-28,19:30,Matchweek 3,Mon,Away,L,0,1,...,NaN,NaN,NaN,NaN,NaN,0,11,19,0.0,0
3,Alaves,2023-2024,2023-09-02,18:30,Matchweek 4,Sat,Home,W,1,0,...,NaN,NaN,NaN,NaN,NaN,1,24,18,5.0,1
4,Alaves,2023-2024,2023-09-15,21:00,Matchweek 5,Fri,Away,L,0,2,...,NaN,NaN,NaN,NaN,NaN,0,20,21,4.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3439,Villarreal,2024-2025,2024-12-08,16:15,Matchweek 16,Sun,Away,L,0,2,...,4.000000,16.400000,0.333333,0.666667,0.666667,0,2,16,6.0,0
3440,Villarreal,2024-2025,2024-12-15,18:30,Matchweek 17,Sun,Home,L,1,2,...,3.666667,16.166667,0.000000,0.333333,0.333333,1,5,18,6.0,0
3441,Villarreal,2024-2025,2024-12-18,21:30,Matchweek 12,Wed,Home,D,1,1,...,2.666667,15.533333,0.000000,0.000000,0.000000,1,20,21,2.0,0
3442,Villarreal,2024-2025,2024-12-22,18:30,Matchweek 18,Sun,Away,W,5,2,...,3.000000,16.366667,0.000000,0.000000,0.000000,0,16,18,6.0,1


In [23]:
# Load 24/25 La Liga data
future_games_path = data_dir / "la_liga_24_25.csv"
future_games = pd.read_csv(future_games_path)

future_games = future_games[(future_games["Wk"] == 19.0) | (future_games["Wk"] == 20.0)]

# Clean Data
future_games = future_games.rename(columns={"Home": "Team", "Away": "Opponent"})

# Drop the first two rows due to played already 
future_games = future_games.iloc[2:].reset_index(drop=True)

# Clean Data
future_games["Team"] = future_games["Team"].str.replace("-", " ")
future_games["Opponent"] = future_games["Opponent"].str.replace("-", " ")

# Clean Data
future_games["Team"] = future_games["Team"].replace({
    "Alavés": "Alaves",
    "Cádiz": "Cadiz",
    "Atlético Madrid": "Atletico Madrid",
    "Leganés" : "Leganes",
    "Betis": "Real Betis"
})
future_games["Opponent"] = future_games["Opponent"].replace({
    "Alavés": "Alaves",
    "Cádiz": "Cadiz",
    "Atlético Madrid": "Atletico Madrid",
    "Leganés" : "Leganes",
    "Betis": "Real Betis"
})

# Save md19_md20.csv
future_games_cleaned_path = output_dir / "md19_md20.csv"
future_games.to_csv(future_games_cleaned_path, index=False)

In [24]:
future_games

,Wk,Day,Date,Time,Venue,Team,Opponent
0,19.0,Fri,2025-01-10,21:00,Estadio del Rayo Vallecano,Rayo Vallecano,Celta Vigo
1,19.0,Sat,2025-01-11,14:00,Estadio de Mendizorroza,Alaves,Girona
2,19.0,Sat,2025-01-11,16:15,Estadio Municipal José Zorrilla,Valladolid,Real Betis
3,19.0,Sat,2025-01-11,18:30,RCDE Stadium,Espanyol,Leganes
4,19.0,Sat,2025-01-11,21:00,Estadio Ramón Sánchez Pizjuán,Sevilla,Valencia
5,19.0,Sun,2025-01-12,13:00,Estadio de Gran Canaria,Las Palmas,Getafe
6,19.0,Sun,2025-01-12,16:15,Riyadh Air Metropolitan Stadium,Atletico Madrid,Osasuna
7,19.0,Mon,2025-01-13,21:00,Reale Arena,Real Sociedad,Villarreal
8,20.0,Fri,2025-01-17,21:00,RCDE Stadium,Espanyol,Valladolid
9,20.0,Sat,2025-01-18,14:00,Estadio El Sadar,Osasuna,Rayo Vallecano


In [27]:
# Feature engineering for future games
future_games["Date"] = pd.to_datetime(future_games["Date"])
future_games["Hour"] = future_games["Time"].str.replace(":.+", "", regex=True)
future_games["Day_Code"] = future_games["Date"].dt.dayofweek
future_games["Venue_Code"] = 1 

# Map Opp_Code manually
opponent_codes = {
    "Real Madrid": 21,
    "Barcelona": 4,
    "Atletico Madrid": 3,
    "Sevilla": 23,
    "Valencia": 24,
    "Celta Vigo": 6,
    "Girona": 12,
    "Real Betis": 5,
    "Leganes": 16, 
    "Getafe": 11,
    "Osasuna": 19,
    "Villarreal": 26,
    "Valladolid": 25,
    "Rayo Vallecano": 20,
    "Athletic Club": 2,
    "Alaves": 0,
    "Las Palmas": 15,
    "Real Sociedad": 22,
    "Mallorca": 18
}
future_games["Opp_Code"] = future_games["Opponent"].map(opponent_codes)

# Save future_games.csv
future_games_path = output_dir / "future_games.csv"
future_games.to_csv(future_games_path, index=False)

In [28]:
future_games

,Wk,Day,Date,Time,Venue,Team,Opponent,Hour,Day_Code,Venue_Code,Opp_Code
0,19.0,Fri,2025-01-10,21:00,Estadio del Rayo Vallecano,Rayo Vallecano,Celta Vigo,21,4,1,6
1,19.0,Sat,2025-01-11,14:00,Estadio de Mendizorroza,Alaves,Girona,14,5,1,12
2,19.0,Sat,2025-01-11,16:15,Estadio Municipal José Zorrilla,Valladolid,Real Betis,16,5,1,5
3,19.0,Sat,2025-01-11,18:30,RCDE Stadium,Espanyol,Leganes,18,5,1,16
4,19.0,Sat,2025-01-11,21:00,Estadio Ramón Sánchez Pizjuán,Sevilla,Valencia,21,5,1,24
5,19.0,Sun,2025-01-12,13:00,Estadio de Gran Canaria,Las Palmas,Getafe,13,6,1,11
6,19.0,Sun,2025-01-12,16:15,Riyadh Air Metropolitan Stadium,Atletico Madrid,Osasuna,16,6,1,19
7,19.0,Mon,2025-01-13,21:00,Reale Arena,Real Sociedad,Villarreal,21,0,1,26
8,20.0,Fri,2025-01-17,21:00,RCDE Stadium,Espanyol,Valladolid,21,4,1,25
9,20.0,Sat,2025-01-18,14:00,Estadio El Sadar,Osasuna,Rayo Vallecano,14,5,1,20
